<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2019-1.</font>
</p>

# Conexiones entre múltiples ventanas  en PyQt

Al crear interfaces con múltiples ventanas, que cambian entre una y otra. Uno puede verse tentado a intentar algo del siguiente estilo, utilizando la interfaz de `show` y `hide`:

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton


class Ventana(QWidget):
    def __init__(self, titulo, x, y, *args):
        super().__init__(*args)
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)

    def abrir_otra_ventana(self):
        self.hide()
        otra_ventana = Ventana("Otra ventana", 300, 100)
        otra_ventana.show()


if __name__ == '__main__':
    a = QApplication(sys.argv)
    ventana = Ventana("Inicial", 100, 100)
    ventana.show()
    sys.exit(a.exec())

Al ejecutarlo, se observa un comportamiento innesperado: **no se muestra la segunda ventana**.
Intentemos un ángulo distinto, instanciemos la segunda ventana antes y la pasamos como un argumento a la ventana inicial:

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton


class Ventana(QWidget):
    def __init__(self, titulo, x, y, otra_ventana=None, *args):
        super().__init__(*args)
        self.otra_ventana = otra_ventana
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)

    def abrir_otra_ventana(self):
        if self.otra_ventana:
            self.hide()
            self.otra_ventana.show()


if __name__ == '__main__':
    a = QApplication(sys.argv)
    otra_ventana = Ventana("Otra ventana", 300, 100)
    ventana = Ventana("Inicial", 100, 100, otra_ventana)
    ventana.show()
    sys.exit(a.exec())

Este sí funciona. ¿Qué ocurrió en el primero entoncee? 

El detalle, es que al instanciar un *widget* como una variable dentro de un método, como toda **variable local** del método, cuando se termine dicho método, Python **descarta** la variable.

```python
def abrir_otra_ventana(self):
    self.hide()
    otra_ventana = Ventana("Otra ventana", 300, 100)
    otra_ventana.show()
```

El método `abrir_otra_ventana` guarda en la variable `otra_ventana` el nuevo *widget* a mostrar. Pero, como `otra_ventana` es una variable local del método, cuando termine de ejecutarse, `otra_ventana` será descartada completamente, eliminando incluso el *widget* recién creado (y mostrado).

La diferencia en el segundo código, es que `otra_ventana` existe fuera de la definición del método, por lo que el hecho de que termine el método, no genera que se descarte la variable y objeto.

Siguiendo esa misma idea, entonces, almacenar la variable `otra_ventana` como un atributo de instancia debería también arreglar el problema. Y es cierto:

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton


class Ventana(QWidget):
    def __init__(self, titulo, x, y, *args):
        super().__init__(*args)
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)

    def abrir_otra_ventana(self):
        self.hide()
        self.otra_ventana = Ventana("Otra ventana", 300, 100)
        self.otra_ventana.show()


if __name__ == '__main__':
    a = QApplication(sys.argv)
    ventana = Ventana("Inicial", 100, 100)
    ventana.show()
    sys.exit(a.exec())

¿Por qué? Porque si se almacena la segunda instancia de ventana como un atributo de la primera, al salir del método `abrir_otra_ventana` no se descarta al objeto ventana; este quedó referenciado en un nivel mayor al método: en la  instancia de la primera ventana..

# ¿Es esa la mejor manera de modelar este comportamiento?

La solución anterior funciona. Y para casos pequeños, basta. El problema es en aplicaciones más grandes. Dónde múltiples ventanas pueden llamarse y hacerse aparecer. Es más, la modelación anterior muestra una relación de **composición** entre las ventanas, es decir, la segunda solo existe bajo la existencia de la primera. Y esta modelación no aplica para todos los casos. 

Múltiples ventanas pueden ser independientes pero aún así puede gatillarse un evento que haga aparecer una desde otra. ¿Cómo puede modelarse esto? 

## ¡Señales!

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton
from PyQt5.QtCore import pyqtSignal

class Ventana(QWidget):
    
    senal_abrir_ventana = pyqtSignal()
    
    def __init__(self, titulo, x, y, *args):
        super().__init__(*args)
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        
        self.senal_abrir_otra_ventana = None
        
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)
        
        self.senal_abrir_ventana.connect(self.show)

    def abrir_otra_ventana(self):
        if self.senal_abrir_otra_ventana:
            self.hide()
            self.senal_abrir_otra_ventana.emit()


if __name__ == '__main__':
    a = QApplication(sys.argv)
    
    # Instanciamos dos ventanas
    ventana_1 = Ventana("Inicial", 100, 100)
    ventana_2 = Ventana("Alternativa", 500, 100)
    
    # Conectamos las señales correspondientes
    ventana_1.senal_abrir_otra_ventana = ventana_2.senal_abrir_ventana
    
    # Con esta también vinculamos que la segunda ventana nos devuelva a la primera
    ventana_2.senal_abrir_otra_ventana = ventana_1.senal_abrir_ventana
    
    ventana_1.show()
    
    sys.exit(a.exec())

Así, cada instancia de `Ventana` es independiente. Es mediante señales que se comunican que una ventana debe aparecer luego de la otra, sin contener directamente una instancia dentro de otra instancia de `Ventana`.